In [2]:
# -*- coding: utf-8 -*-
"""
Processamento de Logs PCIbex - Análise de Sentimentos
Este script processa logs brutos do PCIbex para um experimento de classificação de frases,
extrai informações sobre gênero dos participantes, suas classificações e tempos de resposta,
e combina os resultados com o arquivo de frases randomizadas.
"""

import pandas as pd
import numpy as np
import os
import csv
from collections import Counter

def process_log_file(file_path):
    """
    Processa um arquivo de log do PCIbex e extrai informações relevantes.
    
    Args:
        file_path (str): Caminho para o arquivo de log
        
    Returns:
        tuple: (DataFrame com classificações e tempos, DataFrame com dados de gênero)
    """
    # === 1. Leitura do arquivo CSV ===
    # Pula as primeiras 19 linhas que contêm metadados não relevantes
    df = pd.read_csv(file_path, skiprows=19, header=None)
    
    # Define nomes para as colunas
    df.columns = [
        "ReceptionTime", "ParticipantMD5", "Controller", "ItemNumber", "InnerElementNumber",
        "Label", "Group", "PennElementType", "PennElementName", "Parameter",
        "Value", "EventTime", "Comments"
    ]
    
    # Remove linhas de comentários e labels irrelevantes
    df = df[~df["ReceptionTime"].astype(str).str.startswith("#")].copy()
    df = df[~df["Label"].isin(["TCLE", "instrucoes", "agradecimento"])]
    df.reset_index(drop=True, inplace=True)
    
    # === 2. Extração de dados de gênero ===
    genero_data = df[
        (df["Label"] == "genero") &
        (df["PennElementName"] == "selecionaGenero") &
        (df["Parameter"] == "Selected")
    ][["ParticipantMD5", "Value"]].rename(columns={"Value": "Genero"}).drop_duplicates()
    
    # === 3. Processamento do bloco de frases ===
    # Extrai as classificações (positiva, negativa, neutra)
    selecoes_frases = df[
        (df["Label"] == "frases") &
        (df["Parameter"] == "Selection")
    ].copy()
    
    # Ajusta o ItemNumber (subtrai 3 para alinhar com a numeração correta)
    selecoes_frases["ItemNumber"] = selecoes_frases["ItemNumber"].astype(int) - 3
    selecoes_frases = selecoes_frases[["ParticipantMD5", "ItemNumber", "Value", "EventTime"]]
    selecoes_frases.columns = ["ParticipantMD5", "ItemNumber", "Classificacao", "Timestamp"]
    
    # Obtém tempos de início e fim para calcular o tempo gasto
    start_trials = df[(df["Label"] == "frases") & (df["Parameter"] == "_Trial_") & (df["Value"] == "Start")]
    end_trials = df[(df["Label"] == "frases") & (df["Parameter"] == "_Trial_") & (df["Value"] == "End")]
    
    start_trials = start_trials[["ParticipantMD5", "ItemNumber", "EventTime"]].copy()
    end_trials = end_trials[["ParticipantMD5", "ItemNumber", "EventTime"]].copy()
    
    start_trials["ItemNumber"] = start_trials["ItemNumber"].astype(int) - 3
    end_trials["ItemNumber"] = end_trials["ItemNumber"].astype(int) - 3
    
    start_trials.rename(columns={"EventTime": "StartTime"}, inplace=True)
    end_trials.rename(columns={"EventTime": "EndTime"}, inplace=True)
    
    # Calcula o tempo gasto em cada classificação (em segundos)
    frases_final = selecoes_frases.merge(start_trials, on=["ParticipantMD5", "ItemNumber"], how="left")
    frases_final = frases_final.merge(end_trials, on=["ParticipantMD5", "ItemNumber"], how="left")
    frases_final["Tempo_Gasto"] = ((frases_final["EndTime"] - frases_final["StartTime"]) / 1000).round(3)
    
    frases_final = frases_final[["ParticipantMD5", "ItemNumber", "Classificacao", "Tempo_Gasto", "Timestamp"]]
    
    return frases_final, genero_data

def process_all_logs(log_folder, base_filename="results_prod", num_files=10):
    """
    Processa todos os arquivos de log e seleciona 4 participantes de cada gênero por arquivo.
    
    Args:
        log_folder (str): Pasta contendo os logs brutos
        base_filename (str): Nome base dos arquivos de log
        num_files (int): Número de arquivos numerados a processar
        
    Returns:
        DataFrame: Dados processados de todos os arquivos
    """
    all_processed_data = []
    base_item_number = 0
    
    # Lista de arquivos a processar (arquivo base + numerados)
    files_to_process = [f"{base_filename}.csv"] + [f"{base_filename} ({i}).csv" for i in range(1, num_files)]
    
    for filename in files_to_process:
        file_path = os.path.join(log_folder, filename)
        if not os.path.exists(file_path):
            continue
            
        print(f"Processando arquivo: {filename}")
        
        try:
            # Processa o arquivo atual
            frases, generos = process_log_file(file_path)
            
            # Adiciona offset ao ItemNumber para arquivos subsequentes
            frases["ItemNumber"] = frases["ItemNumber"] + base_item_number
            
            # Codifica o gênero (m/f)
            generos["GeneroCod"] = generos["Genero"].str.lower().map(lambda g: "m" if "masculino" in g else "f")
            
            # Seleciona 4 participantes de cada gênero para este arquivo
            ids_masculinos = generos[generos["GeneroCod"] == "m"].head(4)["ParticipantMD5"].tolist()
            ids_femininos = generos[generos["GeneroCod"] == "f"].head(4)["ParticipantMD5"].tolist()
            ids_selecionados = ids_femininos + ids_masculinos
            
            frases_filtradas = frases[frases["ParticipantMD5"].isin(ids_selecionados)].copy()
            
            # Cria tabelas pivot com identificadores genéricos
            pivot_class = frases_filtradas.pivot_table(
                index="ItemNumber",
                columns="ParticipantMD5",
                values="Classificacao",
                aggfunc="first"
            )
            
            pivot_time = frases_filtradas.pivot_table(
                index="ItemNumber",
                columns="ParticipantMD5",
                values="Tempo_Gasto",
                aggfunc="first"
            )
            
            # Renomeia colunas para identificadores genéricos (f1, f2, m1, m2, etc.)
            fem_cols = [col for col in pivot_class.columns if generos.loc[generos["ParticipantMD5"] == col, "GeneroCod"].iloc[0] == "f"]
            masc_cols = [col for col in pivot_class.columns if generos.loc[generos["ParticipantMD5"] == col, "GeneroCod"].iloc[0] == "m"]
            
            rename_dict_class = {}
            rename_dict_time = {}
            
            for i, col in enumerate(fem_cols[:4], 1):
                rename_dict_class[col] = f"f{i}_class"
                rename_dict_time[col] = f"f{i}_tempo"
                
            for i, col in enumerate(masc_cols[:4], 1):
                rename_dict_class[col] = f"m{i}_class"
                rename_dict_time[col] = f"m{i}_tempo"
                
            pivot_class = pivot_class.rename(columns=rename_dict_class)
            pivot_time = pivot_time.rename(columns=rename_dict_time)
            
            frases_limitadas = pd.concat([pivot_class, pivot_time], axis=1).reset_index()
            
            # Calcula classificações majoritárias
            def majoritaria(row, prefixo):
                colunas = [f"{prefixo}{i}_class" for i in range(1, 5)]
                respostas = [row[col] for col in colunas if pd.notna(row[col])]
                if respostas:
                    return Counter(respostas).most_common(1)[0][0]
                return None
                
            def contar_iguais_majoritaria(row, prefixo, majoritaria):
                colunas = [f"{prefixo}{i}_class" for i in range(1, 5)]
                return sum(1 for col in colunas if pd.notna(row[col]) and row[col] == row[majoritaria])
                
            frases_limitadas["cla_maj_femi"] = frases_limitadas.apply(lambda row: majoritaria(row, "f"), axis=1)
            frases_limitadas["cla_maj_masc"] = frases_limitadas.apply(lambda row: majoritaria(row, "m"), axis=1)
            
            frases_limitadas["qtd_maj_femi"] = frases_limitadas.apply(
                lambda row: contar_iguais_majoritaria(row, "f", "cla_maj_femi"), axis=1
            )
            
            frases_limitadas["qtd_maj_masc"] = frases_limitadas.apply(
                lambda row: contar_iguais_majoritaria(row, "m", "cla_maj_masc"), axis=1
            )
            
            # Organiza as colunas
            colunas_final = [
                "ItemNumber",
                # Majoritárias e quantidades
                "cla_maj_femi", "qtd_maj_femi",
                "cla_maj_masc", "qtd_maj_masc",
                # Classificações femininas
                "f1_class", "f2_class", "f3_class", "f4_class",
                # Classificações masculinas
                "m1_class", "m2_class", "m3_class", "m4_class",
                # Colunas de tempo
                "f1_tempo", "f2_tempo", "f3_tempo", "f4_tempo",
                "m1_tempo", "m2_tempo", "m3_tempo", "m4_tempo"
            ]
            
            frases_formatadas = frases_limitadas[colunas_final]
            all_processed_data.append(frases_formatadas)
            
            # Atualiza base_item_number para o próximo arquivo
            base_item_number = frases["ItemNumber"].max()
            
        except Exception as e:
            print(f"Erro ao processar {filename}: {str(e)}")
            
    if all_processed_data:
        # Combina todos os resultados
        resultado_final = pd.concat(all_processed_data, ignore_index=True)
        return resultado_final
    else:
        print("Nenhum arquivo foi processado com sucesso!")
        return None

def merge_with_phrases(log_df, phrases_file):
    """
    Combina o log processado com o arquivo de frases randomizadas.
    
    Args:
        log_df (DataFrame): DataFrame com os logs processados
        phrases_file (str): Caminho para o arquivo de frases randomizadas
        
    Returns:
        DataFrame: DataFrame combinado com frases e classificações
    """
    # Detecção automática do delimitador do arquivo de frases
    with open(phrases_file, "r", encoding="utf-8") as f:
        sample = f.read(2048)
    
    sniffer = csv.Sniffer()
    dialect = sniffer.sniff(sample)
    
    # Lê o arquivo de frases randomizadas
    df_randomizado = pd.read_csv(phrases_file, sep=dialect.delimiter)
    
    # Remove espaços em branco dos nomes das colunas
    df_randomizado.columns = df_randomizado.columns.str.strip()
    log_df.columns = log_df.columns.str.strip()
    
    # Garante a coluna ID_Random
    if "ID_Random" not in df_randomizado.columns:
        df_randomizado.insert(0, "ID_Random", range(1, len(df_randomizado) + 1))
    
    # Merge pelo índice
    df_log_completo = pd.merge(
        log_df,
        df_randomizado[["ID_Random", "frase"]],
        left_on="ItemNumber",
        right_on="ID_Random",
        how="left"
    )
    
    # Reorganiza as colunas
    colunas = df_log_completo.columns.tolist()
    idx_item = colunas.index("ItemNumber")
    colunas_organizadas = colunas[:idx_item + 1] + ["frase"] + [col for col in colunas if col not in ["ItemNumber", "frase", "ID_Random"]]
    df_log_completo = df_log_completo[colunas_organizadas]
    
    return df_log_completo

def save_csv_with_protected_phrases(df, output_path):
    """
    Salva o DataFrame como CSV com aspas protegendo a coluna de frases.
    
    Args:
        df (DataFrame): DataFrame a ser salvo
        output_path (str): Caminho para o arquivo de saída
    """
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(",".join(df.columns.tolist()) + "\n")
        for _, row in df.iterrows():
            linha = []
            for col in df.columns:
                val = row[col]
                if col == "frase":
                    val = str(val)
                    if not (val.startswith('"') and val.endswith('"')):
                        val = f'"{val}"'
                linha.append(str(val))
            f.write(",".join(linha) + "\n")
    
    print(f"Arquivo salvo em: {output_path}")

def main():
    """
    Função principal que orquestra todo o processamento.
    """
    # Definição de caminhos
    log_folder = "logs/logs_brutos"
    randomizado_path = "dados/MQD_1465_randomizado.csv"
    output_log_path = "dados/MQD_1465_log_original.csv"
    output_final_path = "dados/MQD_1465_log_processado.csv"
    
    # Etapa 1: Processar todos os logs
    print("Etapa 1: Processando logs brutos...")
    resultado_logs = process_all_logs(log_folder)
    
    if resultado_logs is not None:
        # Salva o resultado intermediário (logs processados sem frases)
        resultado_logs.to_csv(output_log_path, index=False)
        print(f"Logs processados salvos em: {output_log_path}")
        
        # Etapa 2: Combinar com o arquivo de frases
        print("\nEtapa 2: Combinando com arquivo de frases randomizadas...")
        resultado_final = merge_with_phrases(resultado_logs, randomizado_path)
        
        # Etapa 3: Salvar o resultado final com frases protegidas
        print("\nEtapa 3: Salvando resultado final...")
        save_csv_with_protected_phrases(resultado_final, output_final_path)
        
        # Exibir estatísticas finais
        print(f"\nProcessamento concluído. Total de {len(resultado_final)} registros processados.")
        print(f"Distribuição de classificações femininas: {resultado_final['cla_maj_femi'].value_counts().to_dict()}")
        print(f"Distribuição de classificações masculinas: {resultado_final['cla_maj_masc'].value_counts().to_dict()}")
    else:
        print("Não foi possível processar os logs. Verifique os arquivos de entrada.")

if __name__ == "__main__":
    main()


Etapa 1: Processando logs brutos...
Processando arquivo: results_prod.csv
Processando arquivo: results_prod (1).csv
Processando arquivo: results_prod (2).csv
Processando arquivo: results_prod (3).csv
Processando arquivo: results_prod (4).csv
Processando arquivo: results_prod (5).csv
Processando arquivo: results_prod (6).csv
Processando arquivo: results_prod (7).csv
Processando arquivo: results_prod (8).csv
Processando arquivo: results_prod (9).csv
Logs processados salvos em: dados/MQD_1465_log_original.csv

Etapa 2: Combinando com arquivo de frases randomizadas...

Etapa 3: Salvando resultado final...
Arquivo salvo em: dados/MQD_1465_log_processado.csv

Processamento concluído. Total de 1465 registros processados.
Distribuição de classificações femininas: {'positiva': 514, 'negativa': 503, 'neutra': 448}
Distribuição de classificações masculinas: {'positiva': 619, 'negativa': 514, 'neutra': 332}
